In [1]:
import os
import re
import cv2
import numpy as np
import SimpleITK as sitk
from tqdm import tqdm
from typing import List, Tuple

# --- МОДЕЛИ ---
from lungmask import mask as lungmask
# Для TotalSegmentator (если хочешь): pip install TotalSegmentator
# from totalsegmentator.python_api import totalseg

# ==== НАСТРОЙКИ ====
input_dirs  = ['norma_png', 'pneumonia_png', 'pneumotorax_png']
output_dirs = ['norma_lung', 'pneumonia_lung', 'pneumotorax_lung']
os.makedirs("temp_seg", exist_ok=True)

# Минимальная площадь маски на срез (в пикселях), чтобы считать "есть грудная клетка"
min_pixels = 5000

# ==== ВСПОМОГАТЕЛЬНОЕ ====
num_re = re.compile(r'(\d+)')

def sort_key(fname: str):
    m = num_re.findall(fname)
    return (int(m[-1]) if m else 0, fname)

def list_pngs(folder: str) -> List[str]:
    files = [f for f in os.listdir(folder) if f.lower().endswith('.png')]
    files.sort(key=sort_key)
    return files

def build_volume(png_dir: str) -> Tuple[np.ndarray, List[str]]:
    files = list_pngs(png_dir)
    if not files:
        return np.zeros((0,512,512), dtype=np.uint8), []
    slices = []
    for f in files:
        p = os.path.join(png_dir, f)
        img = cv2.imread(p, cv2.IMREAD_GRAYSCALE)
        if img is None:
            continue
        if img.shape != (512,512):
            img = cv2.resize(img, (512,512), interpolation=cv2.INTER_NEAREST)
        slices.append(img)
    if not slices:
        return np.zeros((0,512,512), dtype=np.uint8), []
    vol_u8 = np.stack(slices, axis=0)  # (z,y,x)
    return vol_u8, files

def u8_to_hu_approx(vol_u8: np.ndarray, W: float, L: float) -> np.ndarray:
    """
    Обратное окно-приближение: предполагаем, что 0..255 получены из HU окном (W,L)
    с линейным масштабом (без клипа). Тогда HU ≈ (u8/255)*W + (L - W/2).
    """
    return (vol_u8.astype(np.float32) / 255.0) * W + (L - W/2.0)

def sitk_from_hu(vol_hu: np.ndarray) -> sitk.Image:
    vol = vol_hu.astype(np.int16, copy=False)
    img = sitk.GetImageFromArray(vol)  # spacing неизвестен; пусть будет 1
    img.SetSpacing((1.0, 1.0, 1.0))
    return img

def run_lungmask_best(vol_u8: np.ndarray) -> np.ndarray:
    # Пробуем 3 окна — берём маску с максимальной площадью
    windows = [(1500, -600), (1600, -600), (350, 40)]
    best_mask = None
    best_sum = -1
    for W, L in windows:
        hu = u8_to_hu_approx(vol_u8, W, L)
        img = sitk_from_hu(hu)
        try:
            m = lungmask.apply(img)  # (z,y,x), 0/1
            s = m.sum()
            if s > best_sum:
                best_sum = s
                best_mask = m
        except Exception as e:
            # иногда падает на «плоских» томах — просто пытаемся дальше
            continue
    if best_mask is None:
        return np.zeros_like(vol_u8, dtype=np.uint8)
    return best_mask.astype(np.uint8)

import skimage.morphology as morph
import skimage.measure as meas
from scipy.ndimage import binary_fill_holes   # 👈 вот отсюда

def fallback_chest_mask(vol_u8: np.ndarray) -> np.ndarray:
    Z, H, W = vol_u8.shape
    out = np.zeros_like(vol_u8, dtype=np.uint8)
    for i in range(Z):
        sl = vol_u8[i]
        # Бинаризация тела
        thr_body, _ = cv2.threshold(sl, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        body = (sl >= thr_body).astype(np.uint8)

        # Воздух/лёгкие
        thr_lung = int(max(0, thr_body * 0.6))
        lungs = (sl <= thr_lung).astype(np.uint8)
        lungs = cv2.bitwise_and(lungs, body)

        # Морфология
        lungs = morph.binary_opening(lungs, morph.disk(3))
        lungs = morph.binary_closing(lungs, morph.disk(5))
        lungs = morph.remove_small_objects(lungs.astype(bool), 500)

        lab = meas.label(lungs, connectivity=2)
        props = sorted(meas.regionprops(lab), key=lambda p: p.area, reverse=True)
        keep = [p.label for p in props[:3]]
        if keep:
            m = np.isin(lab, keep)
            m = morph.binary_closing(m, morph.disk(7))
            m = binary_fill_holes(m)     # ✅ теперь работает
            out[i] = m.astype(np.uint8)
    return out


def ensure_dir(d):
    os.makedirs(d, exist_ok=True)

def save_selected_slices(src_dir: str, dst_dir: str, files: List[str], vol_u8: np.ndarray, mask: np.ndarray) -> int:
    ensure_dir(dst_dir)
    saved = 0
    for i, fname in enumerate(files):
        if i >= mask.shape[0]:
            break
        if mask[i].sum() >= min_pixels:
            cv2.imwrite(os.path.join(dst_dir, fname), vol_u8[i])
            saved += 1
    return saved

# ==== ОСНОВНОЙ ЦИКЛ ====
for in_dir, out_dir in zip(input_dirs, output_dirs):
    print(f"\n=== {in_dir} → {out_dir} ===")
    total_png = sum(len(fs) for _, _, fs in os.walk(in_dir))
    print(f"Найдено PNG (включая подпапки): {total_png}")

    # Если подкаталоги-исследования, пройдёмся по каждому; если плоско — просто сам in_dir
    subdirs = []
    for root, dirs, files in os.walk(in_dir):
        # Берём только листья, где реально есть PNG
        pngs = [f for f in files if f.lower().endswith('.png')]
        if pngs:
            subdirs.append(root)
    if not subdirs:
        subdirs = [in_dir]

    total_saved = 0
    for study_dir in tqdm(subdirs, desc="Исследования"):
        vol_u8, files = build_volume(study_dir)
        if vol_u8.size == 0 or not files:
            continue

        # 1) Пытаемся lungmask с тремя окнами
        mask = run_lungmask_best(vol_u8)

        # 2) Если маска пустая — фолбэк
        if mask.sum() < min_pixels:
            mask_fb = fallback_chest_mask(vol_u8)
            # Берём лучшую из двух
            if mask_fb.sum() > mask.sum():
                mask = mask_fb

        # 3) Сохраняем только срезы с достаточной площадью маски
        # Сохраняем в подзеркало структуры каталогов относительно out_dir
        rel = os.path.relpath(study_dir, in_dir)
        dst = os.path.join(out_dir, rel)
        saved = save_selected_slices(study_dir, dst, files, vol_u8, mask)
        total_saved += saved

    print(f"Сохранено срезов: {total_saved}")

# Итоговая проверка
for d in output_dirs:
    total_files = sum(len(files) for _, _, files in os.walk(d) if files)
    print(f"Total PNG files in '{d}': {total_files}")



=== norma_png → norma_lung ===
Найдено PNG (включая подпапки): 451


Исследования:   0%|          | 0/1 [00:00<?, ?it/s]
/home/wsl/miniconda3/lib/python3.13/site-packages/tqdm/std.py:636: TqdmWarning: clamping frac to range [0, 1]
102%|██████████| 23/22.55 [00:06<00:00,  3.72it/s]

lungmask 2025-10-01 21:32:36 Postprocessing



100%|██████████| 826/826 [00:18<00:00, 44.50it/s]
102%|██████████| 23/22.55 [00:06<00:00,  3.80it/s]

lungmask 2025-10-01 21:33:09 Postprocessing


...

lungmask 2025-10-01 21:35:06 Postprocessing



100%|██████████| 486/486 [00:10<00:00, 44.82it/s]
101%|██████████| 24/23.8 [00:06<00:00,  3.81it/s]

lungmask 2025-10-01 21:35:33 Postprocessing



0it [00:00, ?it/s]
Исследования: 100%|██████████| 1/1 [01:08<00:00, 68.18s/it]

Сохранено срезов: 458
Total PNG files in 'norma_lung': 374
Total PNG files in 'pneumonia_lung': 368
Total PNG files in 'pneumotorax_lung': 458


In [2]:
for input_dir in input_dirs:
    total_files = sum(len(files) for _, _, files in os.walk(input_dir) if files)
    print(f"Total PNG files in '{input_dir}': {total_files}")

Total PNG files in 'norma_png': 451
Total PNG files in 'pneumonia_png': 367
Total PNG files in 'pneumotorax_png': 476


In [3]:
# посмотрим количество png файлов в каждой из выходных папок
for output_dir in output_dirs:
    total_files = sum(len(files) for _, _, files in os.walk(output_dir) if files)
    print(f"Total PNG files in '{output_dir}': {total_files}")

Total PNG files in 'norma_lung': 374
Total PNG files in 'pneumonia_lung': 368
Total PNG files in 'pneumotorax_lung': 458
